# Setup

In [1]:
from pathlib import Path

In [2]:
Path(".").resolve()

PosixPath('/home/ank/workspace/hub/leoank/chirpminds/chirpminds/notes')

In [5]:
Path(".").resolve().parents[1].joinpath("scratch/dataset/samples/sample_001.mp4").is_file()

True

# Extract frames from video

In [ ]:
from subprocess import run as srun
import shelx

In [ ]:
def extract_frames(in_video: Path, out_dir: Path) -> None:
    cmd = f"ffmpeg -i {in_video.resolve()} -r 2 -s 640x360 -q:v 2 {out_dir.resolve()}/frame%d.jpeg"
    cmd = shelx.splilt(cmd)
    srun(cmd)

In [ ]:
def process_video_files(vid_list: list[Path], out_dir: Path) -> None:
    for vid in vid_list:
        extract_frames(vid, out_dir.joinpath(vid.stem))

# Automatic annotation with Grounded SAM

In [ ]:
from autodistill_grounded_sam_2 import GroundedSAM2
from autodistill.detection import CaptionOntology

# define an ontology to map class names to our GroundedSAM prompt
# the ontology dictionary has the format {caption: class}
# where caption is the prompt sent to the base model, and class is the label that will
# be saved for that caption in the generated annotations
# then, load the model
base_model = GroundedSAM2(
    ontology=CaptionOntology(
        {
            "chickadee": "chickadee",
            "cage": "cage",
            "leaves": "leaves"
        }
    )
)

# label all images in a folder called `context_images`
dataset = base_model.label("./images", extension=".png", output_folder="./labeled")

# Manual annotation refinement

In [ ]:
from imjoy_rpc.hypha.sync import connect_to_server
import asyncio
from skimage import io
import matplotlib.pyplot as plt
import numpy as np

# %%
PORT = "9191"

SERVER_CONFIG = {
  "name": "piximi-server",
  "server_url": f"http://localhost:{PORT}",
  "config": {
      "visibility": "private"
  }
}

# %%
piximi_server = connect_to_server(SERVER_CONFIG)


In [ ]:
# %%
piximi_server.getConnectionInfo()

# %%
token = piximi_server.generate_token()
workspace = piximi_server.getWorkspaceInfo()

# %%
print("Name:", piximi_server.name)
print("workspace", workspace)
print("token", token)

# %%
PIXIMI_URL = "http://localhost:3000"
connection_url = f"{PIXIMI_URL}?hypha_token={token}&hypha_workspace={workspace['name']}"

# %%
# !echo -n "$connection_url" | pbcopy

# %%
# !pbpaste

# %% [markdown]
# In Piximi go to piximi.app followed by the query paramater above, wait for load


In [ ]:
# %%
im = io.imread("/Users/Nodar/Developer/piximi/piximi/src/images/8b_2c_actin-nuclei.tif", plugin="pil")

# %%
io.available_plugins

# %%
im.shape, im.dtype

# %%
plt.imshow(np.stack([(im[0]/im[0].max()*255).astype("uint8"), (im[1]/47*255).astype("uint8"), np.zeros([im.shape[1], im.shape[2]]).astype("uint8")]).transpose([1,2,0]))

# %%
piximi_receive_svc = piximi_server.get_service("piximi-annotator-receiver")

piximi_receive_svc

# %%
piximi_receive_svc.receiveImage(im, "hypha_image")

# %%
received_annotations = []


# %%
def send_annotations(annotations):
    print("Received annotations")
    received_annotations.append(annotations)


# %%
piximi_send_svc = piximi_server.register_service({
    "name": "PIXIMI Annotator send service",
    "id": "piximi-annotator-sender",
    "send_annotations": send_annotations
})


# %% [markdown]
# In Piximi, add annotations to the image, click back, and click save

# %%
len(received_annotations)

# %%
type(received_annotations[0])

# %%
len(received_annotations[0])/im.shape[1]/im.shape[2]

# %%
x = [d for d in received_annotations[0]]

# %%
xx = np.asarray(x).reshape((im.shape[1], im.shape[2], 4))

# %%
plt.imshow(xx)

# %%
plt.imshow(xx[:,:,0] * 127)

# %%
# async def main():
#     server = await connect_to_server({"server_url": "http://localhost:9000"})

#     # Get an existing service
#     # Since "hello-world" is registered as a public service, we can access it using only the name "hello-world"
#     svc = await server.get_service("hello-world")
#     ret = await svc.hello("John")
#     print(ret)

# asyncio.run(main())


# Train object detection model

In [ ]:
from autodistill_yolonas import YOLONAS

target_model = YOLONAS("")
target_model.train("./labeled/data.yaml", epochs=200, device="cuda:0")

# Use trained model for object detection and perform tracking

In [ ]:
import numpy as np
import supervision as sv
from ultralytics import YOLO

segmodel = YOLO("../../runs/segment/train2/weights/best.pt")
tracker = sv.ByteTrack(lost_track_buffer=300)

bounding_box_annotator = sv.BoundingBoxAnnotator()
mask_annotator = sv.MaskAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator(trace_length=500)


def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # results = model(frame)[0]
    resultseg = segmodel(frame)[0]
    segdet = sv.Detections.from_ultralytics(resultseg)
    # dsegdettections = sv.Detections.from_ultralytics(results)
    detections = segdet
    # print(f"Masks: {(detections.mask.size)}")
    detections = tracker.update_with_detections(detections)

    labels = [f"#{tracker_id}" for tracker_id in detections.tracker_id]

    annotated_frame = mask_annotator.annotate(scene=frame.copy(), detections=segdet)
    annotated_frame = bounding_box_annotator.annotate(
        scene=frame.copy(), detections=detections
    )
    annotated_frame = label_annotator.annotate(
        scene=annotated_frame, detections=detections, labels=labels
    )
    annotated_frame = trace_annotator.annotate(
        scene=annotated_frame, detections=detections
    )
    return annotated_frame


sv.process_video(
    source_path="samples/sample_001.mp4",
    target_path="results/sample_003.mp4",
    callback=callback,
)

# Analysis

In [4]:
from IPython.display import Video
Video("../../scratch/dataset/samples/sample_001.mp4", width=1024)